<span style="font-size:2em; color:blue">
    Tema 29: Extensiones sintácticas de Haskell
</span>  

----------

[José A. Alonso](https://www.cs.us.es/~jalonso)  
[Departamento de Ciencias de la Computación e I.A.](https://www.cs.us.es)  
[Universidad de Sevilla](http://www.us.es)  
Sevilla, 25 de agosto de 2019

> __Nota:__ La versión interactiva de este tema se encuentra en [Binder](https://mybinder.org/v2/gh/jaalonso/Temas_interactivos_de_PF_con_Haskell/master?urlpath=lab/tree/temas/Tema-29.ipynb).

En este tema se presenta algunas extensiones de la sintaxis de Haskell no incluidas en los temas anteriores.

# Instalación y uso de la librería de los números naturales

La librería de los números naturales es [nats](http://hackage.haskell.org/package/nats-1.1.2). Se instala con

In [1]:
:! stack install nats

Para usarla, se importa con 

In [2]:
import Numeric.Natural

Un ejemplo de definición usando los naturales es el de la función factorial:

In [3]:
factorial :: Natural -> Natural
factorial 1 = 1
factorial n = n * factorial (n - 1)

y los siguientes son ejemplos de evaluación

In [4]:
factorial 3
factorial 100

6

93326215443944152681699238856266700490715968264381621468592963895217599993229915608941463976156518286253697920827223758251185210916864000000000000000000000000

Se puede comprbar el tipo con

In [5]:
:t factorial
:t factorial 3

factorial :: Natural -> Natural

factorial 3 :: Natural

# Definiciones con 'case'

## Ejemplo: De número a notas

`(nota n)` es la nota correspondiente a la clasificación `n`.

In [6]:
nota :: Int -> String
nota n = 
    case n of
        1 -> "Aprobado"
        2 -> "Notable"
        3 -> "Sobresaliente"
        4 -> "MH"
        _ -> "Desconocido"

Por ejemplo,

In [7]:
nota 3
nota 7

"Sobresaliente"

"Desconocido"

## Ejemplo: Valores de expresiones aritméticas 

`valor op x y` es el valor de la operación `op` (suma o resta) aplicado a los números `x` e `y`.

In [8]:
valor :: Char -> Int -> Int -> Int
valor op x y = 
    case op of
        '+' -> x + y
        '-' -> x - y
        _   -> 0   

Por ejemplo,

In [9]:
valor '+' 2 3
valor '-' 5 2
valor '*' 2 3

5

3

0

# El tipo `Either`

En el `Prelude` está definido el tipo `Either` por

Ejemplo de uso del tipo `Either`: `segundo xs` es el segundo elemento 
de la lista `xs` si existe y si no indica el tipo de la lista.

In [10]:
segundo :: [a] -> Either String a
segundo []      = Left "la lista es vacia"
segundo [_]     = Left "la lista solo tiene un elemento"
segundo (_:x:_) = Right x

Por ejemplo,

In [11]:
segundo [5,3,7]

Right 3

In [12]:
segundo [5,3]

Right 3

In [13]:
segundo [5]

Left "la lista solo tiene un elemento"

In [14]:
segundo []

Left "la lista es vacia"

# Definiciones de tipos con campos

Se considera la siguiente definición del tipo `Usuario`

In [15]:
data Usuario = Usuario Int String String
    deriving Show

y sus correspondiente funciones de acceso

In [16]:
uid :: Usuario -> Int
uid (Usuario i _ _) = i

login :: Usuario -> String
login (Usuario _ l _) = l

password :: Usuario -> String
password (Usuario _ _ p) = p

Se puede definir un usuario por

In [17]:
u1 :: Usuario
u1 = Usuario 1 "Juan" "24082019"

Se pueden calcular sus elementos

In [18]:
uid u1
login u1
password u1

1

"Juan"

"24082019"

Usando campos, la definición del tipo de `Usuario` y sus funciones de acceso se simplifican a

In [19]:
data Usuario' = Usuario' 
    { uid'      :: Int
    , login'    :: String
    , password' :: String 
    }
    deriving Show

La definición de un usuario ahora es

In [20]:
u1' :: Usuario'
u1' = Usuario' { login'    = "Juan"
               , password' = "24092019" 
               , uid'      = 1
               }

El cálculo de sus elementos es

In [21]:
uid' u1'
login' u1'
password' u1'

1

"Juan"

"24092019"

Se pueden usar los campos como patrones. Por ejemplo, para identificar a Juan

In [22]:
esJuan :: Usuario' -> Bool
esJuan Usuario' {login' = x} = x == "Juan"

In [23]:
esJuan u1'

True

Una definición alternativa es

In [24]:
esJuan' :: Usuario' -> Bool
esJuan' Usuario' {login' = "Juan"} = True
esJuan' _                          = False

In [25]:
esJuan' u1'

True

Se puede definir nuevos usuarios modificando campos de otros. Por ejemplo,

In [26]:
u2, u3 :: Usuario'
u2 = u1' {uid' = 2}
u3 = u1' {login' = "Ana"}
u1'
u2
u3

Usuario' {uid' = 1, login' = "Juan", password' = "24092019"}

Usuario' {uid' = 2, login' = "Juan", password' = "24092019"}

Usuario' {uid' = 1, login' = "Ana", password' = "24092019"}

Se pueden combinar tipos con campos. Por ejemplo,

In [27]:
{-# LANGUAGE DuplicateRecordFields #-}

data Persona 
    = Usuario       { uid :: Int, login :: String } 
    | Administrador { aid :: Int, login :: String }

y definir funciones con patrones. Por ejemplo,

In [28]:
login :: Persona -> String  
login (Usuario       _ l) = l
login (Administrador _ l) = l

Se puede aplicar como sigue

In [29]:
p1, p2 :: Persona
p1 = Usuario {uid=1, login="Juan"}
p2 = Administrador {aid=5, login="Ana"}
login p1
login p2

"Juan"

"Ana"

# Uso de `subtract` en lugar de `(-)`

La evaluzción de la siguiente expresión da un error

In [30]:
map (-5) [9,2,7]

: 

Se puede corregir usando `subtract`

In [31]:
map (subtract 5) [9,2,7]

[4,-3,2]

# Aplicación de funciones

## Aplicación de funciones con `($)`

El uso de `($)` permite reducir el número de paréntesis. Por ejemplo, la expresión

In [32]:
length (filter odd (map (div 2) (filter even (map (div 7) [1..5])))) 

1

se puede escribir como

In [33]:
length $ filter odd $ map (div 2) $ filter even $ map (div 7) [1..5]

1

Además, se puede usar junto con zipWith. Por ejemplo,

In [34]:
funs = [(+2), div 7, (*5)]
vals = [20, 30, 40]
zipWith ($) funs vals

[22,0,200]

## Aplicación postfija

El operador para la aplicación postfija es `(&)` que está en la librería `Data.Function`. 

Se importa con

In [35]:
import Data.Function ((&))

La expresión

In [36]:
5 & even

False

es equivalente a

In [37]:
even 5

False

En general, `x & f` es equivalente a `f x`

La expresión

In [38]:
(\xs -> xs ++ reverse xs) [2,1,3] 

[2,1,3,3,1,2]

es eguivalente, aunque con más paréntesis, a

In [39]:
[2,1,3] & \xs -> xs ++ reverse xs

[2,1,3,3,1,2]

# Eliminación de argumentos

Ejemplo de transformación de una definición para eliminar los argumentos

In [40]:
import Data.Char (toUpper)

trans1, trans2, trans3, trans4 :: [String] -> [String]
trans1 l = map (\s -> map toUpper s) (filter (\s -> length s >= 5) l)
trans2 l = map (\s -> map toUpper s) $ filter (\s -> length s >= 5) l
trans3 l = map (map toUpper) $ filter ((>= 5) . length) l
trans4   = map (map toUpper) . filter ((>= 5) . length)

In [41]:
trans1 (words "Hoy es un lunes de agosto")
trans2 (words "Hoy es un lunes de agosto")
trans3 (words "Hoy es un lunes de agosto")
trans4 (words "Hoy es un lunes de agosto")

["LUNES","AGOSTO"]

["LUNES","AGOSTO"]

["LUNES","AGOSTO"]

["LUNES","AGOSTO"]

# Puntos fijos con `fix`

El operador `fix` está definido en la librería `Data.Function` de forma que `fix f` es el menor punto fijo de la función f.

Se importa la función `fix`

In [42]:
import Data.Function (fix)

Se puede definir la longitud usando fix

In [43]:
longitud :: [a] -> Int
longitud = fix $ \f ys -> case ys of
    []     -> 0
    (x:xs) -> 1 + f xs

In [44]:
longitud [4,2,5]

3

Se puede definir el último usando `fix`

In [45]:
ultimo :: [a] -> Maybe a
ultimo = fix $ \f xs ->
    case xs of
        []     -> Nothing
        [x]    -> Just x
        (x:xs) -> f xs

In [46]:
ultimo [3,2,5]
ultimo [3]
ultimo []

Just 5

Just 3

Nothing

# Extensiones

Los pragmas permiten extensiones del lenguaje.

## TupleSections 

Se activa el pragma con

In [47]:
:set -XTupleSections

Ejemplos de uso

In [48]:
map (42,) [1..5]

[(42,1),(42,2),(42,3),(42,4),(42,5)]

In [49]:
map (,42) [1..5]

[(1,42),(2,42),(3,42),(4,42),(5,42)]

In [50]:
map (1, "abc",, 9.2) [3, 5]

[(1,"abc",3,9.2),(1,"abc",5,9.2)]

## LambdaCase

Se activa con

In [51]:
:set -XLambdaCase

Con esta extensión, la anterior definición

In [52]:
nota :: Int -> String
nota n = 
    case n of
        1 -> "Aprobado"
        2 -> "Notable"
        3 -> "Sobresaliente"
        4 -> "MH"
        _ -> "Desconocido"

se puede simplificar a

In [53]:
nota' :: Int -> String
nota' = 
    \case
        1 -> "Aprobado"
        2 -> "Notable"
        3 -> "Sobresaliente"
        4 -> "MH"
        _ -> "Desconocido"

Poe ejemplo,

In [54]:
nota' 3
nota' 7

"Sobresaliente"

"Desconocido"

## ViewPatterns

`dosPalabras s` se verifica si la cadena `s` tiene dos palabras.

In [55]:
dosPalabras :: String -> Bool
dosPalabras s = case words s of
    [_, _] -> True
    _      -> False

Por ejemplo,

In [56]:
dosPalabras "hoy domingo"

True

In [57]:
dosPalabras "hoy es domingo"

False

La definición se puede simplificar con la extensión `ViewPatterns`

In [58]:
{-# LANGUAGE ViewPatterns #-}

dosPalabras' :: String -> Bool
dosPalabras' (words -> [_, _]) = True
dosPalabras' _                 = False

Por ejemplo,

In [59]:
dosPalabras' "hoy domingo"
dosPalabras' "hoy es domingo"

True

False

Otro ejemplo: reconocimiento de las listas que empiezan o terminan en cero. 

In [60]:
empiezaOterminaEnCero :: [Int] -> Bool
empiezaOterminaEnCero (head -> 0) = True
empiezaOterminaEnCero (last -> 0) = True
empiezaOterminaEnCero          _  = False

In [61]:
empiezaOterminaEnCero [0,2,5]
empiezaOterminaEnCero [7,2,0]
empiezaOterminaEnCero [7,2,5]

True

True

False

Ejemplo del factorial con `ViewPatterns`

In [62]:
{-# LANGUAGE ViewPatterns #-}

fact :: Integer -> Integer
fact 0                 = 1
fact (subtract 1 -> n) = (n + 1) * fact n

In [63]:
fact 4

24

## PostfixOperators

Se activa con 

In [64]:
:set -XPostfixOperators

Ejemplo de uso

In [65]:
(!) :: Integer -> Integer
(!) n | n == 0    = 1
      | n >  0    = n * ((n - 1) !)
      | otherwise = error "el número tiene que ser no negativo"

In [66]:
(3 !)

6

## UnicodeSyntax

Se activa con

In [67]:
:set -XUnicodeSyntax

Para usarlo se instala la librería con unicode

In [68]:
:! stack install base-unicode-symbols

Se reinicia el núcleo y se importa la librería unicode de las listas

In [69]:
import Data.List.Unicode 

Ejemplos de uso

In [70]:
2 ∈ [1, 2, 3]
[1, 2, 3] ∪ [1, 3, 5]
[1, 2, 3] ∩ [1, 3, 5]

True

[1,2,3,5]

[1,3]

## MultiWayIf

In [71]:
{-# LANGUAGE MultiWayIf #-}

fn :: Int -> Int -> String
fn x y = if | x == 1    -> "a"
            | y <  2    -> "b"
            | otherwise -> "c"

In [72]:
fn 1 4
fn 2 1
fn 2 3

"a"

"b"

"c"

## DuplicateRecordFields

Se pueden combinar tipos con campos. Por ejemplo,

In [73]:
{-# LANGUAGE DuplicateRecordFields #-}

data Persona 
    = Usuario       { uid :: Int, login :: String } 
    | Administrador { aid :: Int, login :: String }

y definir funciones con patrones. Por ejemplo,

In [74]:
login :: Persona -> String  
login (Usuario       _ l) = l
login (Administrador _ l) = l

Se puede aplicar como sigue

In [75]:
p1, p2 :: Persona
p1 = Usuario {uid=1, login="Juan"}
p2 = Administrador {aid=5, login="Ana"}
login p1
login p2

"Juan"

"Ana"

## RecordWildCards

Los campos son funciones pero con `RecordWildCards` se pueden tratar como valores.

In [76]:
{-# LANGUAGE RecordWildCards #-}

data User = User 
    { uid      :: Int
    , login    :: String
    , password :: String 
    } deriving Show

u1, u2 :: User
u1 = User {uid = 0, login = "Pepe", password = "234"}
u2 = User {uid = 1, login = "Luis", password = "567"}

resumen :: User -> String
resumen User { uid = 0, .. } = "ROOT: " ++ login ++ ", " ++ password
resumen User{ ..}            = login ++ ": " ++ password

In [77]:
resumen u1
resumen u2

"ROOT: Pepe, 234"

"Luis: 567"

Se puede combinar con `ViewPatterns`

In [78]:
{-# LANGUAGE ViewPatterns #-}

invitado :: String -> String -> User
invitado (read -> uid) password = User{ login = "invitado", .. }

In [79]:
u3 :: User
u3 = invitado "3" "789"
u3

User {uid = 3, login = "invitado", password = "789"}

Se puede combinar con `DuplicateRecordFields`

In [80]:
{-# LANGUAGE DuplicateRecordFields #-}

data Persona = Persona { name :: String } deriving Show
data Gato = Gato { name :: String } deriving Show

transforma :: Persona -> Gato
transforma Persona{..} = Gato{..}

In [81]:
transforma $ Persona "Grumpy"

Gato {name = "Grumpy"}

## InstanceSigs

`InstanceSigs`permite escribir signaturas en la declaración de intancias.

In [82]:
{-# LANGUAGE InstanceSigs #-}

data T a = MkT a a

instance Eq a => Eq (T a) where
  (==) :: T a -> T a -> Bool   -- la signatura
  (==) (MkT x1 x2) (MkT y1 y2) = x1==y1 && x2==y2

## GeneralizedNewtypeDeriving

In [83]:
{-# LANGUAGE GeneralizedNewtypeDeriving #-}

newtype Altura = Altura Int
    deriving (Show, Read, Eq, Ord, Num)

# Huecos

Los huecos (con `_`) pueden tener distintos usos.

## Huecos como variable anónima

In [84]:
constante5 :: a -> Int
constante5 _ = 5

Por ejemplo,

In [85]:
constante5 23

5

## Huecos como valores indeterminados

In [86]:
f :: a -> (a,Int)
f x = (_,3)

: 

En el diagnóstico se indica que el hueco tiene que ser de tipo `a`.

## Huecos en la signatura

In [87]:
g :: Int -> _
g = show

: 

En el diagnóstico se indica que el hueco tiene que ser `String`.

# Referencias

+ [Basic syntax extensions](https://www.schoolofhaskell.com/school/to-infinity-and-beyond/pick-of-the-week/guide-to-ghc-extensions/basic-syntax-extensions) de Alexander Altman.
+ [Glasgow Haskell Compiler User's Guide: 13.1. Language options](https://downloads.haskell.org/~ghc/latest/docs/html/users_guide/glasgow_exts.html).
+ [Grokking fix](https://www.parsonsmatt.org/2016/10/26/grokking_fix.html) de Matt Parsons.
+ [Haskell ITMO course at CTD](https://github.com/jagajaga/FP-Course-ITMO) de Dimitry Kovanikov y Arseniy Seroka.
  + [Lecture 2: Basic syntax](http://slides.com/fp-ctd/lecture-2)
  + [Lecture 3: Datas, classes, instances](http://slides.com/fp-ctd/lecture-3).
+ [Pattern and guard extensions](https://www.schoolofhaskell.com/school/to-infinity-and-beyond/pick-of-the-week/guide-to-ghc-extensions/pattern-and-guard-extensions) de Alexander Altman.